In [300]:
import scrapy
import json
import requests
import requests_cache
import numpy as np
import pandas as pd
import seaborn as sns
import urlparse
import csv
import urllib
import os
import os.path
from bs4 import BeautifulSoup
import sys
from lxml import html
import lxml.html as lx

requests_cache.install_cache('demo_cache')

In [63]:
def Popular_Movies_search(year,pages,num):
    count = 0
    names = []
    
    for i in range(1,(pages+1)):
        url = "http://www.imdb.com/search/title?sort=moviemeter,asc&title_type=feature&year=" + str(year) + "," +str(year) + "&" + "page=" + str(i) + "&" + "ref_=adv_prv"        
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        search_results = soup.find_all("p", class_="sort-num_votes-visible")
        vote = int(search_results[0].find_all('span')[1].text.replace(',',''))
        soup = BeautifulSoup(r.content, "html.parser")
        search_results = soup.find_all("div", class_="lister-item-content")
        

        for result in search_results:
            try:
                vote = int(result.find_all('p')[3].find_all('span')[1].text.replace(',',''))
                if vote < 10000:
                    continue
            except:
                continue
            header = result.find("h3", class_="lister-item-header")
            title = header.a
            b = title.get_text().encode('utf-8')
            names.append(b)
            count = count + 1
            if count == num:
                return names
            
    return names

In [321]:
def Movie_info_search(term):
    url = 'http://www.omdbapi.com/?t='+term.replace(' ','+')+'&plot=full&tomatoes=true'
    response = requests.get(url)
    try:
        js = response.json()
        map(js.pop, ['Plot','DVD','tomatoFresh','tomatoConsensus','tomatoMeter','tomatoRating','tomatoRotten','tomatoURL','tomatoUserMeter','tomatoUserRating','tomatoUserReviews','tomatoReviews'])
    except:
        return {}
    meta_url = 'http://www.metacritic.com/movie/'+term.replace(' ','-')
    return js

In [368]:
def Scoring_search(movie):
    imdb = Movie_info_search(movie).values()[-4]
    url = 'http://www.imdb.com/title/' + str(imdb) + '/ratings?ref_=tt_ov_rt'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    res_2 = soup.find_all('td')
    
    rates = []
    try:
        for i in xrange(14):
            number = str(res_2[37 + 3*i].text.replace(u'\xa0',u''))
            rate = str(res_2[38 + 3*i].find_all('img')[0].text.replace(u'\xa0',u''))
            rates.append([number + ',' + rate])
    except:
        print 'sdfdsf'
    
    params = {'males':rates[0], 'females':rates[1],'<18':rates[2], 'males<18':rates[3],'females<18':rates[4], '18-29':rates[5],'males 18-29':rates[6], 'females 18-29':rates[7],'30-44':rates[8],'males 30-44':rates[9], 'females 30-44':rates[10], '45+':rates[11],'males 45+':rates[12], 'females 45+':rates[13]}
    return params

In [369]:
#Scoring_search('la la land')

{'18-29': ['77781,8.6'],
 '30-44': ['42045,8.1'],
 '45+': ['10303,7.8'],
 '<18': ['2086,9.2'],
 'females': ['38798,8.3'],
 'females 18-29': ['22798,8.5'],
 'females 30-44': ['9377,8.1'],
 'females 45+': ['2244,7.7'],
 'females<18': ['651,9.0'],
 'males': ['104358,8.4'],
 'males 18-29': ['54069,8.7'],
 'males 30-44': ['31978,8.1'],
 'males 45+': ['7867,7.8'],
 'males<18': ['1409,9.2']}

In [132]:
def Comment_search(movie,gender):
    comments = []
    imdb = imdb = Movie_info_search(movie).values()[-1]
    url = 'http://www.imdb.com/title/'+str(imdb)+'/reviews?filter=custom&spoiler=&vote_min=&vote_max=&gender='+gender+'&age_min=&age_max=&country=&order=score&custom=1'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    div = soup.find('div', attrs = {'id' : 'tn15content'}).find_all('p')
    for contents in div:
        
        comments.append(contents.text.replace('\n',' '))
    
    return comments
    

In [341]:
films_name = []
films_data = pd.DataFrame()
for year in xrange(0,15):
    films_name.append(Popular_Movies_search(year+2002,10,100))
    films_data = pd.concat([films_data,pd.DataFrame.from_dict(Movie_info_search(i) for i in films_name[year])])

In [367]:
films_data.reset_index(drop = True)
films_data.to_csv('Films_data.csv',encoding = 'utf-8')